In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os


In [ ]:
movies = pd.read_csv('data/tmdb_5000_movies.csv')
credits = pd.read_csv('data/tmdb_5000_credits.csv')

In [ ]:
movies.head(2)

In [ ]:
credits.head(2)

In [ ]:
movies.shape

In [ ]:
credits.shape


In [ ]:
#merging dataset
movies =movies.merge(credits,on = "title")

In [ ]:
movies.head(2)

In [ ]:
movies.shape

In [ ]:
movies['original_language'].value_counts()

In [ ]:
movies.columns

In [ ]:
#selecting important columns only
movies =movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head(2)

In [ ]:
movies.shape

In [ ]:
#checking missing value
movies.isnull().sum()

In [ ]:
#drop missing values
movies.dropna(inplace=True)

In [ ]:
movies.isnull().sum()

In [ ]:
movies.shape

In [ ]:
#checking duplicate values
movies.duplicated().sum()

In [ ]:
# handle genres
movies.iloc[0]['genres']

In [ ]:
import ast #for converting str to list

def convert(text):
    l=[]
    for i in ast.literal_eval(text):
        l.append(i['name'])
        
    return l

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies.head(2)

In [ ]:
movies['keywords']= movies['keywords'].apply(convert)

In [ ]:
movies.head(2)

In [ ]:
movies.iloc[0]['cast']

In [ ]:
def convert_cast(text):
    l=[]
    counter = 0
    for i in ast.literal_eval(text):
        if counter <3:
            l.append(i['name'])
        counter += 1
        
    return l

In [ ]:
movies['cast']= movies['cast'].apply(convert_cast)
movies.head(2)

In [ ]:
movies.iloc[0]['crew']

In [ ]:
def fetch_director(text):
    l=[]
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
        
    return l

In [ ]:
movies['crew']= movies['crew'].apply(fetch_director)
movies.head(2)

In [ ]:
# handle overview (converting to list)

movies.iloc[0]['overview']

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies.head(2)

In [ ]:
# now removing space like that 
'Sam Worthington'
'SamWorthington'

def remove_space(word):
    l=[]
    for i in word:
        l.append(i.replace(" ",""))
    return l


In [ ]:
movies['cast'] = movies['cast'].apply(remove_space)
movies['crew'] = movies['crew'].apply(remove_space)
movies['genre'] = movies['genres'].apply(remove_space)
movies['keywords'] = movies['keywords'].apply(remove_space)

In [ ]:
movies.head(2)

In [ ]:
# Concatinate all
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords']+ movies['cast']+movies['crew']

In [ ]:
movies.head(2)

In [ ]:
movies.iloc[0]['tags']

In [ ]:
# droping those extra columns
new_df = movies[['movie_id','title','tags']]

In [ ]:
new_df.head(2)

In [ ]:
# Converting list to str
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))
new_df.head(2)

In [ ]:
new_df.iloc[0]['tags']

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())
new_df.head(2)

In [ ]:
new_df.iloc[0]['tags']

In [ ]:
import nltk
from nltk.stem import PorterStemmer

In [ ]:
ps = PorterStemmer()

In [ ]:
def stems(text):
    l = []
    for i in text.split():
        l.append(ps.stem(i))
    return " ".join(l)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stems)

In [ ]:
new_df.iloc[0]['tags']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vector

In [ ]:
vector.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
similarity

In [ ]:
similarity.shape

In [ ]:
new_df[new_df['title'] == 'Spider-Man'].index[0]

In [ ]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key= lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('The Dark Knight Rises')

In [ ]:
import pickle
pickle.dump(new_df,open('artifacts/movie_list.pkl','wb'))
pickle.dump(similarity,open('artifacts/similarity.pkl','wb'))